In [1]:
import os


In [2]:
%pwd

'c:\\Users\\chinn\\OneDrive\\Desktop\\projects\\deepL\\Text-Summarizer\\research'

In [3]:
os.chdir(os.path.dirname("../"))

In [31]:
%pwd

'c:\\Users\\chinn\\OneDrive\\Desktop\\projects\\deepL\\Text-Summarizer'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [50]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directory

In [57]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directory([self.config.artifacts_root])
    def get_data_ingestion_config(self)->DataIngestionConfig:
         config=self.config.data_ingestion
         create_directory([config.root_dir])
         data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
         return data_ingestion_config

In [58]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [61]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    def   download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with folloeing info:{headers}")
        else:
            logger.info(f"File already exists of size :{get_size(Path(self.config.local_data_file))})")
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [62]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-13 20:04:26,677: INFO: common: YAML file:config\config.yaml loaded successfully]
[2024-12-13 20:04:26,680: INFO: common: YAML file:params.yaml loaded successfully]
[2024-12-13 20:04:26,682: INFO: common: Created directory: artifacts]
[2024-12-13 20:04:26,684: INFO: common: Created directory: artifacts/data_ingestion]
[2024-12-13 20:06:05,787: INFO: 878171469: artifacts/data_ingestion/data.zip download with folloeing info:Connection: close
Content-Length: 8377572
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aea79eaa36efc9339c42d3a61413f75f62f7e5676002d7473efbe842605ace6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C8EC:42FD4:99EDD:B4832:675C4603
Accept-Ranges: bytes
Date: Fri, 13 Dec 2024 14:34:44 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10239-MAA
X-Cache: MISS
X-Cache-Hit